# MMU walkthrough

This notebook briefly demonstrates the various capabilities of the package

In [1]:
import pandas as pd
import numpy as np
import mmu

## Model generation

We use the model generator to create a dataset of classifiers.

We turn of the model noise as this is not needed at this point.
Additionally, we turn of random train/test splits for each model such
that each model has seen the same data

In [2]:
model = mmu.ModelGenerator(random_state=112356, class_sep=1.0)
fit = model.fit()
train_mask, y, probas, X, models, ground_truth = fit.transform(
    enable_model_noise=False,
    enable_sample_noise=False,
)

Create test sets on which we will compute the metrics

Note that all the functions require contiguous arrays so slicing by column requires F-order

In [3]:
y_test = np.asarray(y[~train_mask, :], order='F')
probas_test = np.asarray(probas[~train_mask, :], order='F')
yhat_test = np.asarray(np.rint(probas_test).astype(np.int64, copy=False), order='F')

### Confusion matrix only

We can compute the confusion matrix for a single run using yhat or based on the probability and a classification threshold

In [4]:
# based on yhat
mmu.confusion_matrix(y_test[:, 0], yhat_test[:, 0])

array([[1482,   42],
       [ 186, 1290]])

In [5]:
# based on proba with 0.5 classification threshold
mmu.confusion_matrix_proba(y_test[:, 0], probas_test[:, 0], threshold=0.5)

array([[1482,   42],
       [ 186, 1290]])

### Confusion matrix and metrics

The ``binary_metrics*`` functions computes ten classification metrics:
 *    0 - neg.precision aka Negative Predictive Value
 *    1 - pos.precision aka Positive Predictive Value
 *    2 - neg.recall aka True Negative Rate & Specificity
 *    3 - pos.recall aka True Positive Rate aka Sensitivity
 *    4 - neg.f1 score
 *    5 - pos.f1 score
 *    6 - False Positive Rate
 *    7 - False Negative Rate
 *    8 - Accuracy
 *    9 - MCC
 
This index can be retrieved using:

In [6]:
col_index = mmu.metrics.col_index
col_index

{'neg.precision': 0,
 'npv': 0,
 'pos.precision': 1,
 'ppv': 1,
 'neg.recall': 2,
 'tnr': 2,
 'specificity': 2,
 'pos.recall': 3,
 'tpr': 3,
 'sensitivity': 3,
 'neg.f1': 4,
 'neg.f1_score': 4,
 'pos.f1': 5,
 'pos.f1_score': 5,
 'fpr': 6,
 'fnr': 7,
 'accuracy': 8,
 'acc': 8,
 'mcc': 9}

### Confusion matrix and binary metrics over a single run

In [7]:
cm, metrics = mmu.binary_metrics(y_test[:, 0], yhat_test[:, 0])

In [8]:
# the confusion matrix
cm

array([[1482,   42],
       [ 186, 1290]])

We can create a dataframe from the confusion matrix using:

In [9]:
mmu.confusion_matrix_to_dataframe(cm)

estimated         
                   negative positive
observed negative      1482       42
         positive       186     1290

In [10]:
# the metrics
metrics

array([0.88848921, 0.96846847, 0.97244094, 0.87398374, 0.92857143,
       0.91880342, 0.02755906, 0.12601626, 0.924     , 0.8516749 ])

We can create a dataframe from the metrics using:

In [11]:
mmu.metrics_to_dataframe(metrics)

,neg.precision,pos.precision,neg.recall,pos.recall,neg.f1,pos.f1,fpr,fnr,acc,mcc
0,0.888489,0.968468,0.972441,0.873984,0.928571,0.918803,0.027559,0.126016,0.924,0.851675


### Confusion matrix and binary metrics over a single run using probabilities

In [12]:
cm, metrics = mmu.binary_metrics_proba(y_test[:, 0], probas_test[:, 0], threshold=0.5)

In [13]:
mmu.confusion_matrix_to_dataframe(cm)

estimated         
                   negative positive
observed negative      1482       42
         positive       186     1290

In [14]:
mmu.metrics_to_dataframe(metrics)

,neg.precision,pos.precision,neg.recall,pos.recall,neg.f1,pos.f1,fpr,fnr,acc,mcc
0,0.888489,0.968468,0.972441,0.873984,0.928571,0.918803,0.027559,0.126016,0.924,0.851675


### Confusion matrix and binary metrics over a single run using multiple thresholds

In [15]:
thresholds = np.linspace(1e-5, 1.0, 1000)

In [16]:
cm, metrics = mmu.binary_metrics_thresholds(
    y=y_test[:, 0],
    proba=probas_test[:, 0],
    thresholds=thresholds,
    fill=1.0
)

In [17]:
cm

array([[   0, 1524,    0, 1476],
       [ 131, 1393,    4, 1472],
       [ 275, 1249,    5, 1471],
       ...,
       [1520,    4, 1030,  446],
       [1520,    4, 1114,  362],
       [1524,    0, 1476,    0]])

In [18]:
mmu.metrics_to_dataframe(metrics)

,neg.precision,pos.precision,neg.recall,pos.recall,neg.f1,pos.f1,fpr,fnr,acc,mcc
0,1.000000,0.492000,0.000000,1.000000,0.000000,0.659517,1.000000,0.000000,0.492000,1.000000
1,0.970370,0.513787,0.085958,0.997290,0.157926,0.678185,0.914042,0.002710,0.534333,0.200761
2,0.982143,0.540809,0.180446,0.996612,0.304878,0.701144,0.819554,0.003388,0.582000,0.304291
3,0.983516,0.557663,0.234908,0.995935,0.379237,0.714981,0.765092,0.004065,0.609333,0.353451
4,0.985075,0.580403,0.303150,0.995257,0.463623,0.733217,0.696850,0.004743,0.643667,0.410783
...,...,...,...,...,...,...,...,...,...,...
995,0.616917,0.987061,0.995407,0.361789,0.761737,0.529499,0.004593,0.638211,0.683667,0.464476
996,0.607772,0.986111,0.995407,0.336721,0.754726,0.502020,0.004593,0.663279,0.671333,0.444123
997,0.596078,0.991111,0.997375,0.302168,0.746195,0.463136,0.002625,0.697832,0.655333,0.419391
998,0.577069,0.989071,0.997375,0.245257,0.731121,0.393051,0.002625,0.754743,0.627333,0.370627


### Confusion matrix and binary metrics over a multiple runs using multiple thresholds

In [19]:
cm, metrics = mmu.binary_metrics_runs_thresholds(
    y=y_test,
    proba=probas_test,
    thresholds=thresholds,
    fill=1.0
)

The confusion matrix and metrics are now cubes.

For the confusion matrix the:
* row -- over the thresholds
* colomns -- the confusion matrix elements
* slices -- the runs

The stride is such that the biggest stride is over the thresholds for the confusion matrix and over the metrics for the metrics.

The argument being that you will want to model the confusion matrices over the runs
and the metrics individually over the thresholds and runs

In [20]:
print('shape confusion matrix: ', cm.shape)
print('strides confusion matrix: ', cm.strides)

shape confusion matrix:  (1000, 4, 30)
strides confusion matrix:  (960, 8, 32)


In [21]:
print('shape metrics: ', metrics.shape)
print('strides metrics: ', metrics.strides)

shape metrics:  (1000, 10, 30)
strides metrics:  (240, 240000, 8)


### Binary metrics over confusion matrices

In [22]:
# We use binary_metrics_thresholds to create confusion matrices
cm, _ = mmu.binary_metrics_thresholds(
    y=y_test[:, 0],
    proba=probas_test[:, 0],
    thresholds=thresholds,
    fill=0.0
)

In [23]:
metrics = mmu.binary_metrics_confusion(cm, 0.0)

In [24]:
mmu.metrics_to_dataframe(metrics)

,neg.precision,pos.precision,neg.recall,pos.recall,neg.f1,pos.f1,fpr,fnr,acc,mcc
0,0.000000,0.492000,0.000000,1.000000,0.000000,0.659517,1.000000,0.000000,0.492000,0.000000
1,0.970370,0.513787,0.085958,0.997290,0.157926,0.678185,0.914042,0.002710,0.534333,0.200761
2,0.982143,0.540809,0.180446,0.996612,0.304878,0.701144,0.819554,0.003388,0.582000,0.304291
3,0.983516,0.557663,0.234908,0.995935,0.379237,0.714981,0.765092,0.004065,0.609333,0.353451
4,0.985075,0.580403,0.303150,0.995257,0.463623,0.733217,0.696850,0.004743,0.643667,0.410783
...,...,...,...,...,...,...,...,...,...,...
995,0.616917,0.987061,0.995407,0.361789,0.761737,0.529499,0.004593,0.638211,0.683667,0.464476
996,0.607772,0.986111,0.995407,0.336721,0.754726,0.502020,0.004593,0.663279,0.671333,0.444123
997,0.596078,0.991111,0.997375,0.302168,0.746195,0.463136,0.002625,0.697832,0.655333,0.419391
998,0.577069,0.989071,0.997375,0.245257,0.731121,0.393051,0.002625,0.754743,0.627333,0.370627
